# CS 105 Spring 2022 OH/recitation scheduling

## Inputs
* Office hour block schedule.
* Recitation schedule.
* Availability form data.
* TA metadata (returning status).
* TA recitation buddy data (derived from availability form).

## IP formulation (informal)

### Objective (summary)
Jointly maximize utility-weighted TA hours and utility-weighted recitation assignments, with a utility bonus for recitation buddy pairings.

### Hard constraints
* Office hour shifts are between 1 and 3 hours long.
* Each TA is scheduled only when available.
* Each TA works at most one office hour shift per day.
* Each TA works at most one maximum-length (3hr) shift per week.
* Each TA works no more than 3 office hour shifts per week.
* Each TA with a recitation preference score of ≥4 holds no more than 2 recitations per week.
* Each TA with a recitation preference score of 1-3 holds no more than 1 recitation per week.
* Each TA with a recitation preference score of 0 holds no recitations.
* No TA can simultaneously hold office hours and lead a recitation.
* No TA has more than one weekend shift.
* To avoid excessive late-night trips trips to Cummings, no TA has a shift starting after 9 PM.
* Estimated demand is (roughly) met at each block: 0-1 TAs in blocks with demand 1, and 1-2 TAs in blocks with demand 2. (Blocks are an hour long.) 
* Each recitation has between 1 and 2 leaders.
* A minimum number of TA-hours are assigned globally.
* Each TA is assigned a number of hours in the range they asked for. Each recitation counts as 2 hours, and weekly grading parties are counted as 2 hours. (Being conservative and counting them as 3 hours makes scheduling much hairier for spring 2022 data, and grading parties are almost always ≤2 hours long anyway. Furthermore, not all weeks have a grading party.)

### Utility considerations
* **OH block objective**: A TA gets 4 points of utility when granted a preferred office hour block and 1 point when granted a non-preferred block.
* **Recitation schedule objective**: A TA gets $1.6^{r}$ units of utility when granted a recitation, where $r$ is their recitation preference score from 1-5. (TAs with a score of 0 are not scheduled for recitations.) This utility score is multiplied by 2 for returning TAs, which encourages solutions where returning TAs hold more recitations and fewer office hours.
* **Recitation buddy objective**: We add 5 points for each directional recitation match. Thus, if two TAs mutually request to hold recitation together, they each get 5 points of utility for each recitation they hold together.

We'll get a first-pass solution with a MIP solver. (I originally tried Z3, but it doesn't do well with complicated objective functions.) Some manual tweaks will be necessary, of course. In particular, assigning in-person vs. virtual office hours is probably a judgment call.

### Notes for future semesters
In addition to the fields on the [existing form](https://forms.gle/aPt4M3fWpQwxpdvH9), it would be ideal to gather:
* Would you rather have shorter shifts throughout the week or one long shift? are you willing to do a 3-hour shift?
* How many recitations are you willing to lead?
* Is there anyone you want to be paired with for office hours?
* Is there anyone you _don't_ want to be paired with for office hours or recitations? 😳


## Running
This notebook is mostly standalone. CSV inputs are omitted from the GitHub repository for privacy reasons. The schema for `ta_buddies.csv` is just pairs of TA names (`requester`, `requested`). The schema for `ta_metadata.csv` is `name` and `returning` (a boolean). The schema for `availability.csv` (the main input) is [induced by this Google Form](https://forms.gle/aPt4M3fWpQwxpdvH9). **Names must match exactly between CSVs.**

In [ ]:
!pip install pandas ortools python-dateutil

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
from collections import namedtuple, defaultdict
from dateutil.parser import parse as ts_parse
from ortools.linear_solver import pywraplp

## Model parameters

In [ ]:
max_oh_shift_length = 3    # in hours
max_oh_shifts_per_ta = 3
max_recitations_per_ta = 2
recitation_time_commitment = 2  # in hours (prep + session), approx.
grading_time_commitment = 2  # in hours (typical; being more conservative makes schedulning significantly harder)
one_shift_per_day = True  # optional constraint: at most one shift per TA per day (no extra walks!)
oh_preferred_weight = 4  # multiplier in objective function for preferred hours (non-preferred hours have unit weight)
recitation_pref_shape = 1.4  # base preference score is `recitation_pref_shape**[1 - 5 ranking]`
recitation_returning_ta_multiplier = 1.8  # prefer senior TAs as recitation leaders
recitation_one_way_buddy_weight = 5  # add extra utility for buddy pairings
alpha = 5  # relative weight of recitation objective (w.r.t. OH block objective, which has unit weight)
beta = 5   # relative weight of OH shift objective

## OH schedule

In [ ]:
weekdays = ('Mon', 'Tue', 'Wed', 'Thu', 'Fri')
weekend = ('Sat', 'Sun')

### Guidance from Richard
  > For coverage insight, look at the pinned piazza post from last semester; we're looking to roughly mirror that schedule, with more coverage on Monday/Tuesday nights (leading up to major deadlines), and less on Thursdays. We do have many more students this semester, though, so we could probably handle ~15% more hours across the board if possible.
  
> No one shows up in the morning, so those never need double coverage (in my experience).

> We were well under budget this past semester (woo-hoo!), hitting about 63 office hours per week for 90 students. This semester we have 120 students enrolled currently, so we could bump up to ~80 hours per week, I believe, and still be kosher. If you can get between 63-80 hours per week, I think that's a great start.
  

### Last semester (fall 2021)

#### Weekdays
* Monday, 10 AM – 1 PM (1 TA; last hour virtual)
* Monday, 3 PM – 7 PM (1 TA)
* Monday, 7 PM – 12 PM (2 TAs)
* Tuesday, 10 AM – 11 AM (1 TA)
* Tuesday, 12 PM – 3 PM (1 TA, first hour virtual)
* Tuesday, 4 PM – 8 PM (1 TA)
* Tuesday, 8 PM – 12 AM (2 TAs)
* Wednesday, 10 AM – 1 PM (1 TA)
* Wednesday, 3 PM – 9 PM (1 TA)
* Wednesday, 9 PM – 12 AM (2 TAs)
* Thursday, 10 AM – 7 PM (1 TA)
* Thursday, 8 PM – 10 PM (1 TA)
* Friday, 11 AM – 3 PM (1 TA)

#### Weekend
* Saturday, 2 PM – 7 PM (1 TA)
* Sunday, 1 PM – 5 PM (1 TA)
* Sunday, 6 PM – 8 PM (1 TA)
* Sunday, 8 PM – 9 PM (2 TAs)
* Sunday, 9 PM – 10 PM (1 TA)

In [ ]:
oh_demand_bounds = {  # in TAs
  ('Mon', 10, 17): (0, 1),  # Monday, 10 AM – 6 PM
  ('Mon', 18, 23): (1, 2),  # Monday, 6 PM – 12 AM
  ('Tue', 10, 15): (1, 1),  # Tuesday, 10 AM – 4 PM
  ('Tue', 16, 23): (2, 2),  # Tuesday, 4 PM – 12 AM
  ('Wed', 12, 23): (1, 1),  # Wednesday, 12 PM – 12 AM
  ('Thu', 12, 20): (0, 1),  # Thursday, 12 PM – 9 PM
  ('Fri', 10, 14): (0, 1),  # Friday, 10 AM – 3 PM  [grading party at 3 PM]
  ('Sat', 12, 18): (1, 1),  # Saturday, 12 PM – 7 PM
  ('Sun', 12, 21): (1, 1),  # Sunday, 12 PM – 10 PM
}

In [ ]:
print('Estimated hours of demand (unweighted):\t', sum(end - start + 1 for _, start, end in oh_demand_bounds))
print('Estimated TA-hours of demand (u.b.):\t', sum((end - start + 1) * weight for (_, start, end), (_, weight) in oh_demand_bounds.items()))

In [ ]:
 # this seems reasonable given the bounds above...
min_total_ta_hours = 80

In [ ]:
Block = namedtuple('Block', ['day', 'start', 'end'])

In [ ]:
oh_block_ids = {}
oh_demand_by_block_id = {}
for (day, start, end), demand in oh_demand_bounds.items():
  for hour in range(start, end + 1):
    block = Block(day, hour, hour + 1)
    block_id = len(oh_block_ids)
    oh_block_ids[block] = block_id
    oh_demand_by_block_id[block_id] = demand
oh_block_id_to_block = {v: k for k, v in oh_block_ids.items()}

## Recitation schedule

In [ ]:
Recitation = namedtuple('Recitation', ['day', 'start', 'end'])

In [ ]:
def parse_recitation(line):
  line = line.replace('Th', 'Thu').replace('Fr', 'Fri')
  day, start_raw, _, end_raw = line.split(' ')
  start = ts_parse(start_raw)
  end = ts_parse(end_raw)
  return Recitation(day,
                    start.hour + (start.minute / 60),
                    end.hour + (end.minute / 60))

In [ ]:
# from SIS; same format used in Google Forms
recitations_raw = """
Th 10:30AM - 11:45AM
Th 12:00PM - 1:15PM
Th 1:30PM - 2:45PM
Th 3:00PM - 4:15PM
Th 4:30PM - 5:45PM
Th 6:00PM - 7:15PM
Th 7:30PM - 8:45PM
Fr 10:30AM - 11:45AM
Fr 12:00PM - 1:15PM
Fr 1:30PM - 2:45PM""".strip().split('\n')

recitations = [parse_recitation(line) for line in recitations_raw]

## Availability
(Raw CSV export from Google Forms.)

In [ ]:
availability_df = pd.read_csv('availability.csv')  # excluded from repo for privacy reasons

In [ ]:
columns = {
  'Name': 'name',
  'Minimum hours per week': 'min_hours',
  'Maximum hours per week': 'max_hours',
  'What is your preferred modality for office hours?': 'in_person_pref',
  'How willing are you to lead recitation?': 'recitation_pref',
  'If your answer to the above was greater than 0, which recitations would you want to lead.': 'recitation_availability'
}

In [ ]:
for col in availability_df.columns:
  if col.startswith('Time Availability.') or col.startswith('Select high preference'):
    hour = ts_parse(col.split('[')[-1].replace(']', '')).hour
    prefix = 'oh_available' if col.startswith('Time Availability.') else 'oh_pref'
    columns[col] = f'{prefix}_{hour}'

In [ ]:
availability_df = availability_df.rename(columns=columns)[columns.values()]

In [ ]:
def embed_block_schedule(row, prefix='oh_available'):
  row = dict(row)
  schedule = np.zeros(len(oh_block_ids), dtype=bool)
  for col in row:
    if col.startswith(prefix) and col != prefix:
      block_hour = int(col.split('_')[-1])
      block_days = str(row[col]).split(';')
      for day in block_days:
        block = Block(day[:3], block_hour, block_hour + 1)
        if block in oh_block_ids:
          schedule[oh_block_ids[block]] = 1
  return schedule

In [ ]:
availability_df['oh_available'] = availability_df.apply(embed_block_schedule, axis=1)
availability_df['oh_pref_available'] = availability_df.apply(partial(embed_block_schedule, prefix='oh_pref'), axis=1)

In [ ]:
def embed_recitations(row):
  schedule = np.zeros(len(recitations), dtype=bool)
  row_recitations = str(row['recitation_availability']).split(';')
  for recitation in row_recitations:
    if recitation in recitations_raw:
      schedule[recitations_raw.index(recitation)] = 1
  return schedule

In [ ]:
availability_df['recitation_available'] = availability_df.apply(embed_recitations, axis=1)

In [ ]:
availability_df['name'] = availability_df['name'].str.strip()

In [ ]:
availability_df = availability_df[['name', 'min_hours', 'max_hours', 'recitation_pref',
                                   'oh_available', 'oh_pref_available', 'recitation_available']]

In [ ]:
ta_metadata_df = pd.read_csv('ta_metadata.csv', dtype={'returning': bool})

In [ ]:
availability_df = availability_df.merge(ta_metadata_df, on='name', how='left')

In [ ]:
availability_df

## Input data cleanup/validation
* If a TA is unwilling to lead a recitation, they should not have any recitation availability.
* If a TA has a preferred time checked, they should also be available for that time. (preferred => available, i.e. ~(preferred $\land$ ~available))
* TAs who are unavailable for anything other than grading should be filtered out.

In [ ]:
no_recitation_tas_df = availability_df[availability_df['recitation_pref'] == 0]
assert (no_recitation_tas_df['recitation_available'].sum() == 0).all()

In [ ]:
# Special cases...
availability_df.at[2, 'max_hours'] = 5
availability_df.at[9, 'oh_available'] = availability_df.loc[9, 'oh_pref_available'] 

In [ ]:
availability_df.apply(lambda row: (row['oh_pref_available'] & ~row['oh_available']).sum(), axis=1)

In [ ]:
availability_df = availability_df[availability_df['max_hours'] > grading_time_commitment].reset_index(drop=True)

In [ ]:
ta_buddies_df = pd.read_csv('ta_buddies.csv')

In [ ]:
buddies = []
names = list(availability_df['name'])
for requester, requested in zip(ta_buddies_df['requester'], ta_buddies_df['requested']):
  if requester in names and requested in names:
    buddies.append((names.index(requester), names.index(requested)))

In [ ]:
buddies

In [ ]:
availability_df

## IP model: constraints

In [ ]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### OH availability constraints
We only create boolean block/shift variables when TAs are available.

In [ ]:
Shift = namedtuple('Shift', ['var', 'day', 'start', 'end'])

In [ ]:
oh_blocks = [{} for _ in range(len(oh_block_ids))]
oh_blocks_by_ta = []
oh_shifts_by_ta = []
for ta_idx, availability in enumerate(availability_df['oh_available']):
  row_blocks = {}
  #print('availability:', availability)
  for block_id in np.where(availability == 1)[0]:
    #print('creating', ta_idx, block_id)
    # Constraint (implicit): only generate variables for TAs when they are available.
    block_var = solver.IntVar(0, 1, f'ta_{ta_idx}_block_{block_id}')
    row_blocks[block_id] = block_var
    oh_blocks[block_id][ta_idx] = block_var
  
  row_shifts = []
  block_var_to_shift_vars = defaultdict(list)
  for window in range(max_oh_shift_length):
    for block_idx in range(availability.size - window):
      start_block = oh_block_id_to_block[block_idx]
      end_block = oh_block_id_to_block[block_idx + window]
      
      # Constraint: no short shifts at night/when classes have ended.
      if start_block.start >= 21 and window < 2:  
        continue
        
      # Assumption: within a day, blocks are contiguous.
      if ((start_block.day == end_block.day)
          and (availability[block_idx:block_idx + window + 1].sum() == window + 1)):
        shift_var = solver.IntVar(0, 1, f'ta_{ta_idx}_shift_{len(row_shifts)}')
        row_shifts.append(Shift(shift_var, start_block.day, start_block.start, end_block.end))
        
        # Constraint: if the shift is selected, all blocks in the shift are selected.
        for block_id in range(block_idx, block_idx + window + 1):
          block_var = row_blocks[block_id]
          solver.Add(block_var >= shift_var)
          block_var_to_shift_vars[block_var].append(shift_var)
          
  # Constraint: if a block is selected, then a shift it is in is selected.
  for block_var, shift_vars in block_var_to_shift_vars.items():
    #print(block_var, shift_vars)
    solver.Add(solver.Sum(shift_vars) >= block_var)
    
  # Constraint: orphan blocks (blocks outside of a valid shift) cannot be assigned.
  for orphan_block_var in set(row_blocks.values()) - set(block_var_to_shift_vars):
    solver.Add(orphan_block_var == 0)

  if one_shift_per_day:
    # Constraint (optional): no overlapping blocks (or one shift per day).
    shift_vars_by_day = defaultdict(set)
    for shift in row_shifts:
      shift_vars_by_day[shift.day].add(shift.var)
    #print(shift_vars_by_day)
    for day_vars in shift_vars_by_day.values():
      solver.Add(solver.Sum(day_vars) <= 1)
  else:
    # TODO: intra-day shift conflict constraints.
    raise NotImplementedError('Multiple office hour shifts per day not supported yet.')
    
  # Constraint: no more than k shifts per week.
  solver.Add(solver.Sum(s.var for s in row_shifts) <= max_oh_shifts_per_ta)
  
  # Constraint: no more than one shift per week of maximum length.
  long_shift_vars = [
    shift.var for shift in row_shifts
    if shift.end - shift.start == max_oh_shift_length
  ]
  solver.Add(solver.Sum(long_shift_vars) <= 1)
    
  # Constraint: nobody has more than one weekend shift.
  weekend_shift_vars = [
    shift.var for shift in row_shifts
    if shift.day in weekend
  ]
  solver.Add(solver.Sum(weekend_shift_vars) <= 1)
          
  oh_blocks_by_ta.append(row_blocks)
  oh_shifts_by_ta.append(row_shifts)

In [ ]:
# Constraint: demand met at each block.
for block_id, ta_vars in enumerate(oh_blocks):
  demand_lb, demand_ub = oh_demand_by_block_id[block_id]
  solver.Add(demand_lb <= solver.Sum(ta_vars.values()) <= demand_ub)

### Recitation availability constraints
* **Supply side**: TAs can only be assigned to recitations they might be available for.
* **Supply side**: Each TA cannot lead too many recitations.
* **Demand side**: each recitation must have at between one and two leaders.

In [ ]:
recitation_assignments_by_ta = []
recitation_assignments = [{} for _ in range(len(recitations))]
for ta_idx, row in enumerate(availability_df.itertuples()):
  availability = row.recitation_available
  recitation_pref = row.recitation_pref
  row_recitations = {}
  for recitation_id in np.where(availability == 1)[0]:
    # Constraint (implicit): only generate variables for TAs when they are available.
    ta_recitation_var = solver.IntVar(0, 1, f'ta_{ta_idx}_recitation_{recitation_id}')
    row_recitations[recitation_id] = ta_recitation_var
    recitation_assignments[recitation_id][ta_idx] = ta_recitation_var    
  recitation_assignments_by_ta.append(row_recitations)
  
  # Constraint: each TA with a preference score of ≥4 leads ≤k recitations.
  # Constraint: each TA with a preference score between 1 and 3 leads ≤1 recitations.
  threshold = max_recitations_per_ta  if recitation_pref >= 4 else 1
  solver.Add(solver.Sum(row_recitations.values()) <= threshold)

In [ ]:
# more bespoke constraints...
solver.Add(recitation_assignments_by_ta[6][0] == 1)
solver.Add(recitation_assignments_by_ta[6][4] == 0)

In [ ]:
for ta_vars in recitation_assignments:
  # Constraint: each recitation must have ≥1 leader.
  # Constraint: each recitation must have ≤2 leaders.
  solver.Add(1 <= solver.Sum(ta_vars.values()) <= 2)

In [ ]:
# bespoke constraints to reflect ancillary preferences...
solver.Add(recitation_assignments_by_ta[22][5] == 1)
solver.Add(recitation_assignments_by_ta[22][7] == 1)
solver.Add(oh_blocks_by_ta[0][0] == 1) 
solver.Add(oh_blocks_by_ta[0][1] == 1) 
solver.Add(oh_blocks_by_ta[0][2] == 1) 
solver.Add(solver.Sum(oh_blocks_by_ta[0].values()) == 3) 

### Recitation/OH overlap constraints
A TA cannot simultaneously lead a recitation and hold office hours.

In [ ]:
print('Overlaps:')
for recitation_id, recitation in enumerate(recitations):
  for block, block_id in oh_block_ids.items():
    if (block.day == recitation.day and
        max(recitation.start, block.start) < min(recitation.end, block.end)):
      print(block, recitation, end=' ')
      for ta_idx, ta_blocks in enumerate(oh_blocks_by_ta):
        if block_id in ta_blocks and recitation_id in recitation_assignments_by_ta[ta_idx]:
          print(ta_idx, end=' ')
          block_var = ta_blocks[block_id]
          recitation_var = recitation_assignments_by_ta[ta_idx][recitation_id]
          solver.Add(block_var + recitation_var <= 1)
      print()

### Per-TA commitment constraints
A TA's expected weekly hour commitment (between grading, recitation, and office hours) should be within their desired range.

In [ ]:
for row in availability_df.itertuples():
  commitment_vars = (
    [(v, 1) for v in oh_blocks_by_ta[row.Index].values()] +
    [(v, recitation_time_commitment)
     for v in recitation_assignments_by_ta[row.Index].values()]
  )
  commitment_sum = solver.Sum(v * weight for v, weight in commitment_vars)
  solver.Add(commitment_sum <= row.max_hours - grading_time_commitment)
  if row.min_hours > grading_time_commitment:
    solver.Add(commitment_sum >= row.min_hours - grading_time_commitment)

### Minimum time commitment constraints

In [ ]:
all_block_vars = []
for ta_vars in oh_blocks:
  all_block_vars += list(ta_vars.values())
solver.Add(solver.Sum(all_block_vars) >= min_total_ta_hours)

## IP model: objective function

### Office hours utility

In [ ]:
oh_utility_obj_clauses = []
for row in availability_df.itertuples():
  ta_idx = row.Index
  availability = row.oh_available
  pref_availability = row.oh_pref_available
  for block_id in np.where(availability == 1)[0]:
    block_var = oh_blocks_by_ta[ta_idx][block_id]
    weight = oh_preferred_weight if pref_availability[block_id] == 1 else 1
    oh_utility_obj_clauses.append(weight * block_var)
    
oh_utility_obj = solver.Sum(oh_utility_obj_clauses)

### Recitation utility (base)

In [ ]:
recitation_utility_obj_clauses = []
for recitation_vars, row in zip(recitation_assignments_by_ta, availability_df.itertuples()):
  recitation_pref = row.recitation_pref
  returning_mult = recitation_returning_ta_multiplier if row.returning else 1
  ta_utility = int(round(returning_mult * recitation_pref_shape**recitation_pref))
  for var in recitation_vars.values():
    recitation_utility_obj_clauses.append(ta_utility * var)

### Recitation utility (buddies)

For a pair of recitation decision variables $x_{ij}, x_{ik}$, we form a "buddy variable" $y_{ijk}$ such that $y_{ijk} = x_{ij} \land x_{ik}$. We can do this by forcing $x_{ij} + x_{ik} -1 \leq y_{ijk} \leq x_{ij} + x_{ik}$.

In [ ]:
for requester_id, requested_id in buddies:
  requester_recitations = set(recitation_assignments_by_ta[requester_id])
  requested_recitations = set(recitation_assignments_by_ta[requested_id])
  for shared_recitation in requester_recitations & requested_recitations:
    requester_var = recitation_assignments[shared_recitation][requester_id]
    requested_var = recitation_assignments[shared_recitation][requested_id]
    buddy_var = solver.IntVar(0, 1, f'buddy_{requester_id}_{requested_id}_{shared_recitation}')
    solver.Add(requester_var + requester_var - 1 <= buddy_var <= requester_var + requester_var)
    recitation_utility_obj_clauses.append(recitation_one_way_buddy_weight * buddy_var)

In [ ]:
recitation_utility_obj = solver.Sum(recitation_utility_obj_clauses)

In [ ]:
total_obj = oh_utility_obj + alpha * recitation_utility_obj

In [ ]:
solver.Maximize(total_obj)
solver.Solve()

In [ ]:
pretty_oh_table = defaultdict(lambda: {day: [] for day in list(weekdays) + list(weekend)})
for block_id, ta_vars in enumerate(oh_blocks):
  for ta_id, ta_var in ta_vars.items():
    if ta_var.solution_value() > 0.5:
      block = oh_block_id_to_block[block_id]
      ta_name = availability_df.loc[ta_id, 'name']
      hour = block.start - 12 if block.start > 12 else block.start
      suffix = 'PM' if block.start >= 12 else 'AM'
      pretty_oh_table[f'{hour}:00 {suffix}'][block.day].append(ta_name)
      print(block, ta_name)

In [ ]:
oh_solution_df = pd.DataFrame(pretty_oh_table).transpose()
for col in oh_solution_df.columns:
  oh_solution_df[col] = oh_solution_df[col].apply(lambda v: ', '.join(v))
oh_solution_df.to_csv('oh_solution_v2.csv')

In [ ]:
oh_solution_df

In [ ]:
recitation_solution = defaultdict(list)
for recitation_id, ta_vars in enumerate(recitation_assignments):
  for ta_id, ta_var in ta_vars.items():
    if ta_var.solution_value() > 0.5:
      recitation = recitations[recitation_id]
      ta_name = availability_df.loc[ta_id, 'name']
      recitation_solution[recitation_id].append(ta_name)
      print(recitation, ta_name)

In [ ]:
pretty_recitation_table = [
  {
    'Recitation': recitations_raw[recitation_id],
    'TAs': ', '.join(ta_names)
  }
  for recitation_id, ta_names in recitation_solution.items()
]
pd.DataFrame(pretty_recitation_table).to_csv('recitation_solution_v2.csv', index=False)

In [ ]:
pd.DataFrame(pretty_recitation_table)

### 